### Explanation of the problem

si - service intention (which train, for now just one train in toy example)  
r - route (path in the graph)  
rs - route section (edges of the graph)  
tin - time into a route section  
tou - time out of a route section  
M - a large number for linearisation of constraints  
EarIn - earliest time into a route section  
EarOut - earliest time out of a route section  
$\delta_{si, r}$ = 1 if route chosen is to be used, and 0 otherwise  
$x_{si, rs}$ = 1 if route section is to be used, and 0 otherwise  
mrt - the minimum running time at an edge for a train  
mst - the minimum stopping time at an edge for a train

## Constraints for the problem

time into a section comes before time out: 


(1)  $t_{si,r,rs}^{in} \leq t_{si,r,rs}^{out}$  

time into next section comes after time out from previous section:  
(2)  $t_{si,r,rs}^{out} \leq t_{si,r,rs+1}^{in}$

if r is the selected path, satisfy minimum running and stopping time:  
(3)  $t_{si,r,rs}^{out} - t_{si,r,rs}^{in} \geq mrt_{si, r, rs} + mst_{si, r, rs} - M(1-\delta_{si,r})$

Earliest in constraint:  
(4)  $t_{si,r,rs}^{in} \geq EarIn_{si,r,rs} - M(1-\delta_{si,r})$  

Earliest out constraint:  
(5)  $t_{si,r,rs}^{out} \geq EarOut_{si,r,rs} - M(1-\delta_{si,r})$

Select only one path to every service:  
(6)  $\sum_{r \in P} \delta_{si,r} = 1$  

All route sections on the chosen track are occupied  
(7)  $x_{si, re} \geq \delta_{si, r}$


### Programme constraints, add into a variable called cs

In [7]:
import cvxpy as cp
tin = [cp.Variable(nonneg = True)]*4
tout = [cp.Variable(nonneg = True)]*4
xs = [cp.Variable(boolean = True)]*3
deltas = [cp.Variable(boolean = True)]*2


cs = []


#1
cs.append((tin[0] <= tout[0]))
cs.append((tin[1] <= tout[1]))
cs.append((tin[2] <= tout[2]))
cs.append((tin[3] <= tout[3]))

#2
cs.append((tout[0] <= tin[1]))
cs.append((tout[2] <= tin[3]))

#3
M=2000
mrt = 53
mst = 0
cs.append((tout[0] - tin[0] >= mrt + mst - M*(1-deltas[0])))
mrt = 32
mst = 0
cs.append((tout[1] - tin[1] >= mrt + mst - M*(1-deltas[0])))
mrt = 53
mst = 0
cs.append((tout[2] - tin[2] >= mrt + mst - M*(1-deltas[1])))
mrt = 2000
mst = 0
cs.append((tout[3] - tin[3] >= mrt + mst - M*(1-deltas[1])))

#4
EarIn = 28200
cs.append((tin[0] >= EarIn - M*(1-deltas[0])))
cs.append((tin[2] >= EarIn - M*(1-deltas[1])))

#5
#--

#6
cs.append((deltas[0] + deltas[1] == 1))


#7
cs.append((xs[0] >= deltas[0]))
cs.append((xs[1] >= deltas[0]))
cs.append((xs[0] >= deltas[1]))
cs.append((xs[2] >= deltas[1]))

In [8]:
prob = cp.Problem(cp.Minimize(0),
                 cs)
prob.solve(solver = 'MOSEK', verbose = True)

                                     CVXPY                                     
                                    v1.1.14                                    
(CVXPY) Aug 08 02:08:10 PM: Your problem has 4 variables, 17 constraints, and 0 parameters.
(CVXPY) Aug 08 02:08:10 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 08 02:08:10 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 08 02:08:10 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 08 02:08:10 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Aug 08 02:08:10 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -

inf

### View variable values

In [ ]:
for tag,v in zip(['tin', 'tout', 'x', 'delta'],[tin, tout, xs, deltas]):
    for e in v:
        print(tag, e.value)

### Mosek says the prolem is infeasible, but

### Proof that there is a feasible integer solution:

In [5]:
#Proposed solution
tin = [28200, 28253, 28200, 28253]
tout = [28253, 28285, 28253, 28253+2000]
deltas = [1, 0]
xs = [1,1,0]


cs = []


#1
cs.append((tin[0] <= tout[0]))
cs.append((tin[1] <= tout[1]))
cs.append((tin[2] <= tout[2]))
cs.append((tin[3] <= tout[3]))

#2
cs.append((tout[0] <= tin[1]))
cs.append((tout[2] <= tin[3]))

#3
M=2000
mrt = 53
mst = 0
cs.append((tout[0] - tin[0] >= mrt + mst - M*(1-deltas[0])))
mrt = 32
mst = 0
cs.append((tout[1] - tin[1] >= mrt + mst - M*(1-deltas[0])))
mrt = 53
mst = 0
cs.append((tout[2] - tin[2] >= mrt + mst - M*(1-deltas[1])))
mrt = 2000
mst = 0
cs.append((tout[3] - tin[3] >= mrt + mst - M*(1-deltas[1])))

#4
EarIn = 28200
cs.append((tin[0] >= EarIn - M*(1-deltas[0])))
cs.append((tin[2] >= EarIn - M*(1-deltas[1])))

#5
#--

#6
cs.append((deltas[0] + deltas[1] == 1))


#7
cs.append((xs[0] >= deltas[0]))
cs.append((xs[1] >= deltas[0]))
cs.append((xs[0] >= deltas[1]))
cs.append((xs[2] >= deltas[1]))

All constraints come out as true for the proposed solution:

In [6]:
cs

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

### What is wrong here? Why does MOSEK say problem is infeasible